In [1]:
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
reso_properties = pd.read_csv('data/reso_reso_properties_icaar.csv')

/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_61558/3690172652.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reso_properties = pd.read_csv('data/reso_reso_properties_icaar.csv')


In [3]:
event_codes = {
    'Coming Soon': 50,
    'Active': 100,
    'Contingent': 200,
    'Active Under Contract': 240,
    'Pending': 240,
    'Expired': 300,
    "Incomplete": 999,
    'Hold': 640,
    'Closed': 500,
    "Withdrawn": 600,
    "Canceled": 620,
    "Delete": 600
}

# Exploring the Data
Getting the counts by status for Buckets
- check to see which dates are duplicates
- break down by available dates

In [4]:
reso_properties.columns

Index(['listing_id', 'standard_status', 'cancellation_date', 'close_date',
       'contingent_date', 'contract_status_change_date', 'expiration_date',
       'land_lease_expiration_date', 'listing_contract_date',
       'off_market_date', 'on_market_date', 'purchase_contract_date',
       'withdrawn_date', 'updated_at', 'furnished_availability_date',
       'vendor_ignore_update_y_n', 'tenant_lease_expires_date',
       'estimated_close_date', 'close_price', 'furnished_list_price',
       'list_price', 'list_price_low', 'original_list_price',
       'previous_list_price', 'price_change_timestamp'],
      dtype='object')

In [5]:
len(reso_properties)

83961

In [6]:
r_count = reso_properties.count()
r_count = r_count[r_count > 0]
r_count

listing_id                83961
standard_status           83961
close_date                47976
expiration_date           83961
listing_contract_date     83961
off_market_date           81803
purchase_contract_date    48065
updated_at                83961
close_price               52238
list_price                78838
original_list_price       78050
price_change_timestamp    83961
dtype: int64

In [8]:
reso_properties = reso_properties[r_count.index]
reso_properties.head()

,listing_id,standard_status,close_date,expiration_date,listing_contract_date,off_market_date,purchase_contract_date,updated_at,close_price,list_price,original_list_price,price_change_timestamp
0,20112554,Expired,NaN,2011-10-13,2011-04-14,2011-10-14,NaN,2024-01-23 03:02:56.219234,NaN,524000.0,524000.0,2011-04-15 04:00:00
1,20106967,Expired,NaN,2011-09-30,2010-12-28,2011-10-01,NaN,2024-01-23 03:02:45.48096,NaN,587400.0,775000.0,2011-05-31 04:00:00
2,20105947,Expired,NaN,2011-10-08,2010-10-08,2011-10-09,NaN,2024-01-23 03:02:45.483022,NaN,55900.0,55900.0,2010-10-08 04:00:00
3,20105946,Expired,NaN,2011-10-08,2010-10-08,2011-10-09,NaN,2024-01-23 03:02:45.266369,NaN,54900.0,54900.0,2010-10-08 04:00:00
4,20084442,Expired,NaN,2011-12-31,2008-07-18,2012-01-01,NaN,2024-01-22 23:33:40.184876,NaN,55000.0,55000.0,2008-07-22 04:00:00


In [14]:
reso_properties.groupby('standard_status').count().to_csv('buckets_with_price_date_distribution.csv')

In [59]:
def coalesce(*values):
    """Return the first non-None value or None if all values are None"""
    return next((v for v in values if pd.notna(v)), None)

def create_histories_buckets(status, listing_contract_date, list_price, original_list_price, purchase_contract_date, close_date, close_price, expiration_date, off_market_date):
    match status:
        case 'Active':
            events = [(100, listing_contract_date, coalesce(original_list_price, list_price, close_price))]
        case 'Closed':
            events = [(100, listing_contract_date, coalesce(original_list_price, list_price, close_price)),
                      (240, purchase_contract_date, list_price),
                      (500, close_date, coalesce(close_price, list_price))]
        case 'Expired':
            events = [(100, listing_contract_date, coalesce(original_list_price, list_price, close_price)),
                      (300, expiration_date, coalesce(list_price, close_price))]
        case 'Pending':
            events = [(100, listing_contract_date, coalesce(original_list_price, list_price, close_price)),
                      (240, purchase_contract_date, coalesce(list_price, close_price))]
        case 'Withdrawn':
            events = [(100, listing_contract_date, coalesce(original_list_price, list_price, close_price)),
                      (600, off_market_date, coalesce(list_price, close_price))]
            
    try:
        return events
    except UnboundLocalError:
        print(status)

In [60]:
reso_properties['events'] = [create_histories_buckets(a, b, c, d, e, f, g, i, j) for a, b, c, d, e, f, g, i, j in zip(reso_properties['standard_status'], reso_properties['listing_contract_date'], reso_properties['list_price'], reso_properties['original_list_price'], reso_properties['purchase_contract_date'], reso_properties['close_date'], reso_properties['close_price'], reso_properties['expiration_date'], reso_properties['off_market_date'])]

In [61]:
def is_valid(status, events):
    try:
        for e in events:
            if None in e:
                return False
        if events[-1][0] != event_codes[status]:
            return False
        return True
    except:
        print(status, events, type(events))

In [62]:
reso_properties['is_valid'] = [is_valid(x, y) for x, y in zip(reso_properties['standard_status'], reso_properties['events'])]

In [63]:
reso_properties.to_csv('data/reso_properties_event_codes.csv', index=False)

In [64]:
len(reso_properties[~reso_properties['is_valid']])

585

In [65]:
len(reso_properties)

83961